In [20]:
import os
import json

In [21]:
# construct skeleton JSON to populate with your own data
datalist_json = {
    "testing": [],
    "training": []
}

In [22]:
#populate testing with files in your directory
test_dir = 'data/imagesTs/'

for file in os.listdir(test_dir):
    datalist_json["testing"].append({
        "image": './imagesTs/' + file,
    })

In [23]:
datalist_json

{'testing': [{'image': './imagesTs/amos.nii.gz'},
  {'image': './imagesTs/amos_0013.nii.gz'},
  {'image': './imagesTs/amos_0018.nii.gz'},
  {'image': './imagesTs/amos_0022.nii.gz'},
  {'image': './imagesTs/amos_0029.nii.gz'},
  {'image': './imagesTs/amos_0032.nii.gz'},
  {'image': './imagesTs/amos_0034.nii.gz'},
  {'image': './imagesTs/amos_0040.nii.gz'},
  {'image': './imagesTs/amos_0041.nii.gz'},
  {'image': './imagesTs/amos_0051.nii.gz'},
  {'image': './imagesTs/amos_0056.nii.gz'},
  {'image': './imagesTs/amos_0061.nii.gz'},
  {'image': './imagesTs/amos_0063.nii.gz'},
  {'image': './imagesTs/amos_0070.nii.gz'},
  {'image': './imagesTs/amos_0073.nii.gz'},
  {'image': './imagesTs/amos_0085.nii.gz'},
  {'image': './imagesTs/amos_0087.nii.gz'},
  {'image': './imagesTs/amos_0090.nii.gz'},
  {'image': './imagesTs/amos_0106.nii.gz'},
  {'image': './imagesTs/amos_0108.nii.gz'},
  {'image': './imagesTs/amos_0112.nii.gz'},
  {'image': './imagesTs/amos_0117.nii.gz'},
  {'image': './imagesTs/am

In [24]:
#populate training with files in your directory
train_dir = 'data/imagesTr/'
label_dir = 'data/labelsTr/'

for file in os.listdir(train_dir):
    datalist_json["training"].append({
        "image": './imagesTr/' + file,
        "label": './labelsTr/' + file,
        "fold": 0 # initialize as single fold
    })

In [25]:
datalist_json

{'testing': [{'image': './imagesTs/amos.nii.gz'},
  {'image': './imagesTs/amos_0013.nii.gz'},
  {'image': './imagesTs/amos_0018.nii.gz'},
  {'image': './imagesTs/amos_0022.nii.gz'},
  {'image': './imagesTs/amos_0029.nii.gz'},
  {'image': './imagesTs/amos_0032.nii.gz'},
  {'image': './imagesTs/amos_0034.nii.gz'},
  {'image': './imagesTs/amos_0040.nii.gz'},
  {'image': './imagesTs/amos_0041.nii.gz'},
  {'image': './imagesTs/amos_0051.nii.gz'},
  {'image': './imagesTs/amos_0056.nii.gz'},
  {'image': './imagesTs/amos_0061.nii.gz'},
  {'image': './imagesTs/amos_0063.nii.gz'},
  {'image': './imagesTs/amos_0070.nii.gz'},
  {'image': './imagesTs/amos_0073.nii.gz'},
  {'image': './imagesTs/amos_0085.nii.gz'},
  {'image': './imagesTs/amos_0087.nii.gz'},
  {'image': './imagesTs/amos_0090.nii.gz'},
  {'image': './imagesTs/amos_0106.nii.gz'},
  {'image': './imagesTs/amos_0108.nii.gz'},
  {'image': './imagesTs/amos_0112.nii.gz'},
  {'image': './imagesTs/amos_0117.nii.gz'},
  {'image': './imagesTs/am

In [26]:
#split training into 5 folds randomly
import random
random.seed(42)

for i in range(5):
    for j in range(len(datalist_json["training"])):
        if random.random() < 0.2:
            datalist_json["training"][j]["fold"] = i

In [27]:
datalist_json

{'testing': [{'image': './imagesTs/amos.nii.gz'},
  {'image': './imagesTs/amos_0013.nii.gz'},
  {'image': './imagesTs/amos_0018.nii.gz'},
  {'image': './imagesTs/amos_0022.nii.gz'},
  {'image': './imagesTs/amos_0029.nii.gz'},
  {'image': './imagesTs/amos_0032.nii.gz'},
  {'image': './imagesTs/amos_0034.nii.gz'},
  {'image': './imagesTs/amos_0040.nii.gz'},
  {'image': './imagesTs/amos_0041.nii.gz'},
  {'image': './imagesTs/amos_0051.nii.gz'},
  {'image': './imagesTs/amos_0056.nii.gz'},
  {'image': './imagesTs/amos_0061.nii.gz'},
  {'image': './imagesTs/amos_0063.nii.gz'},
  {'image': './imagesTs/amos_0070.nii.gz'},
  {'image': './imagesTs/amos_0073.nii.gz'},
  {'image': './imagesTs/amos_0085.nii.gz'},
  {'image': './imagesTs/amos_0087.nii.gz'},
  {'image': './imagesTs/amos_0090.nii.gz'},
  {'image': './imagesTs/amos_0106.nii.gz'},
  {'image': './imagesTs/amos_0108.nii.gz'},
  {'image': './imagesTs/amos_0112.nii.gz'},
  {'image': './imagesTs/amos_0117.nii.gz'},
  {'image': './imagesTs/am

In [29]:
with open('datalist.json', 'w', encoding='utf-8') as f:
    json.dump(datalist_json, f, ensure_ascii=False, indent=4)